### Write a function to fech data.

In [10]:
# Importing libraries

import os
import sys
import pandas as pd
import numpy as np
import warnings 
import scipy.stats as scs
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [3]:
# Suppressing warning messages

warnings.filterwarnings('ignore')

In [4]:
# setting path to folders

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
scripts_dir = parent_dir+"/scripts"
data_dir = parent_dir+"/data"

sys.path.insert(1, scripts_dir)

In [12]:
# loading script

from data_summarizing_functions import DataSummarizer
from processing_functions import ConditionalSPRT
from dashboard_viz import VizManager
from processing_functions import DataProcessor

sumar = DataSummarizer()
viz = VizManager()
DP = DataProcessor()

In [6]:
# loading data

clean_df = pd.read_csv(data_dir+"/SmartAd_clean_data.csv")
clean_df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
1,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1,0
2,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,6,Chrome Mobile,0,1
3,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1,0
4,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,1


In [13]:
# Giving parameters and finding out the result of the sequential testing

alpha = 0.5
beta = 0.10
odd_ratio = 1.01
c_sprt = ConditionalSPRT()
pipe = Pipeline(
    steps=[
        ("use_custome_cleaner", FunctionTransformer(DP.clean_missing)),
        ("transform data", FunctionTransformer(DP.transform_data)),
        ("ab testing", FunctionTransformer(c_sprt.conditionalSPRT, 
            kw_args={"t1": odd_ratio, "alpha": alpha, "beta": beta})),      
        ])
res = pipe.fit_transform(clean_df)

Missing values:  0
Usable rows:  1243
dataframe splitted
dataframe aggregated on datetime
bernaulli series generated
control df received 586
exposed df received 657
or, alpha, beta:  1.01 0.5 0.1
stats generated


In [14]:
# Printing the output of the test

print(c_sprt.get_output(res))

{'Test': 'Sequential AB testing', 'outcome': 'Unable to conclude.Needs more sample.', 'numberOfObservation': 586}


In [ ]:
def ConditionalSPRT(self,x,y,t1,alpha=0.05,beta=0.10,stop=None):
  """
   #
  # Meeker's SPRT for matched `x` (treatment) and `y` (control), 
  # both indicator responses, likelihood ratio t1, error rates alpha and beta,
  # and (optionally) truncation after trial stop.
  #
  # The return variable contains these elements:
  #(outcome,n, k,l,u,truncated,truncate_decision,x1,r,stats,limits)
  # * outcome:   "continue," "reject null," or "accept null".
  # * n: number observation used for the decsion
  # * k:     Index at which the outcome decision was made (or NA)
  # * l:     lower critical point
  # * u:     upper critical point
  # * truncate_decision: The approximate decision made after truncate point
  # * truncated: If the test was truncated, the value of `n.0`; NA otherwise
  # * x1:       Original data `x`, cumulative
  # * r:         Cumulative sum of x+y
  # * stats:     Series of cumulative sums of log probability ratios
  # * limits:    Two rows giving lower and upper critical limits, respectively
  #
  """
        if t1<=1:
            printLog('warning',"Odd ratio should exceed 1.")
        if (alpha >0.5) | (beta >0.5):
            printLog('warning',"Unrealistic values of alpha or beta were passed."
                     +" You should have good reason to use large alpha & beta values")
        if stop!=None:
            stop=math.floor(n0)

        def comb(n, k):
            return factorial(n) // factorial(k) // factorial(n - k)
        
        def lchoose(b, j):
            a=[]
            if (type(j) is list) | (isinstance(j,np.ndarray)==True):
                if len(j)<2:
                    j=j[0]
            if (type(j) is list) | (isinstance(j,np.ndarray)==True):
                for k in j:
                    n=b
                    if (0 <= k) & (k<= n):
                        a.append(math.log(comb(n,k)))
                    else:
                        a.append(0)
            else:
                n=b
                k=j
                if (0 <= k) & (k<= n):
                    a.append(math.log(comb(n,k)))
                else:
                    a.append(0)

            return np.array(a)

        def g(x,r,n,t1,t0=1):
          """
         #
          # Meeker's (1981) function `g`, the log probability ratio.
          # 
          """
            return -math.log(h(x,r,n,t1))+math.log(h(x,r,n,t0))

        def h(x,r,n,t=1):
          """
          #
          # Reciprocal of Meeker's (1981) function `h`: the conditional probability of 
          # `x` given `r` and `n`, when the odds ratio is `t`.
          #
          # `x` is his "x1", the number of positives in `n` control trials.
          # `r` is the total number of positives.
          # `n` is the number of (control, treatment) pairs.
          # `t` is the odds ratio.
          #
          """
            return f(r,n,t,offset=ftermlog(x,r,n,t))

        def f(r,n,t,offset=0):
          """#
          # Meeker's (1981) function exp(F(r,n,t)), proportional to the probability of 
          #  `r` (=x1+x2) in `n` paired trials with an odds ratio of `t`.
          #
          # This function does *not* vectorize over its arguments.
          #"""
            upper=max(0,r-n)
            lower=min(n,r)
            rng=list(range(upper,lower+1))
            return np.sum(fterm(rng,r,n,t,offset))

        def fterm(j,r,n,t,offset=0):
            ftlog=ftermlog(j,r,n,t,offset)
            return np.array([math.exp(ex) for ex in ftlog])

        def ftermlog(j,r,n,t,offset=0):
          """
          #
          # Up to an additive constant, the log probability that (x1, x1+x2) = (j, r) 
          # in `n` paired trials with odds ratio of `t`.
          #
          # `offset` is used to adjust the result to avoid under/overflow.
          #
          """
            xx=r-j
            lch=lchoose(n,j)
            lchdiff=lchoose(n,xx)
            lg=np.array(j)*math.log(t)
            lgsum=lch+lchdiff
            lgsum2=lgsum+lg
            lgdiff=lgsum2-offset

            return lgdiff

        def logf(r,n,t,offset=0):
          """
          #
          # A protected vesion of log(f), Meeker's function `F`.
          #
          """
            z=f(r,n,t,offset)
            if z>0:
                return math.log(z)
            else:
                return np.nan

        def clowerUpper(r,n,t1c,t0=1,alpha=0.05,beta=0.10):
          """
           #
          # Meeker's (1981) functions c_L(r,n) and c_U(r,n), the  critical values for x1.
          # 0 <= r <= 2n; t1 >= t0 > 0.
          #
          """
            offset=ftermlog(math.ceil(r/2),r,n,t1c)
            z=logf(r,n,t1c,logf(r,n,t0,offset)+offset)
            a=-math.log(alpha/(1-beta))
            b=math.log(beta/(1-alpha))
            lower=b
            upper=1+a
            return (np.array([lower,upper])+z)/math.log(t1c/t0)

        l=math.log(beta/(1-alpha))
        u=-math.log(alpha/(1-beta))
        sample_size=min(len(x),len(y))
        n=np.array(range(1,sample_size+1))

        if stop!=None:
            n=np.array([z for z in n if z<=stop])
        x1=np.cumsum(x[n-1])
        r=x1+np.cumsum(y[n-1])
        stats=np.array(list(map(g,x1, r, n, [t1]*len(x1)))) #recurcively calls g
         #
          # Perform the test by finding the first index, if any, at which `stats`
          # falls outside the open interval (l, u).
          #
        clu=list(map(clowerUpper,r,n,[t1]*len(r),[1]*len(r),[alpha]*len(r), [beta]*len(r)))
        limits=[]
        for v in clu:
            inArray=[]
            for vin in v:
                inArray.append(math.floor(vin))
            limits.append(np.array(inArray))
        limits=np.array(limits)

        k=np.where((stats>=u) | (stats<=l))
        cvalues=stats[k]
        if cvalues.shape[0]<1:
            k= np.nan
            outcome='Unable to conclude.Needs more sample.'
        else:
            k=np.min(k)
            if stats[k]>=u:
                outcome=f'Exposed group produced a statistically significant increase.'
            else:
                outcome='Their is no statistically significant difference between two test groups'
        if (stop!=None) & (k==np.nan):
          #
          # Truncate at trial stop, using Meeker's H0-conservative formula (2.2).
          # Leave k=NA to indicate the decision was made due to truncation.
          #
            c1=clowerUpper(r,stop,t1,alpha,beta)
            c1=math.floor(np.mean(c1)-0.5)
            if x1[n0]<=c1:
                truncate_decision='h0'
                outcome='Maximum Limit Decision. The aproximate decision point shows their is no statistically significant difference between two test groups'
            else:
                truncate_decision='h1'
                outcome=f'Maximum Limit Decision. The aproximate decision point shows exposed group produced a statistically significant increase.'
            truncated=stop
        else:
            truncate_decision='Non'
            truncated=np.nan
        return (outcome,n, k,l,u,truncated,truncate_decision,x1,r,stats,limits)